In [1]:
from typing import List, Dict, Iterable, Tuple, Callable
from matplotlib import pyplot as plt
from collections import Counter
# import pygal
import sys
import os
import string
import urllib.request
import requests
import curl
import pycurl
import webbrowser
import numpy as np
import math
import pandas as pd
# from IPython import qt
from matplotlib.pyplot import figure
from py.xml import raw
from requests.api import get
from matplotlib import pyplot as plt
# from scratch.working_with_data import rescale
# from scratch.multiple_regression import least_squares_fit, predict
# from scratch.gradient_descent import gradient_step

# from stats import mean, median, de_mean, standard_deviation, correlation
# from gradient_descent import minimize_stochastic, maximize_stochastic, maximize_batch
# from vector import dot, vector_add
# from normal import normal_cdf
# from matrix import make_matrix, get_column, shape, matrix_multiply
# from logistic_regression import *

import math
import os
import random
import sys
from functools import partial, reduce

from scipy.optimize import fmin_tnc

import tqdm

from typing import*

from collections import*
# from scipy import*
from sklearn.metrics import*

from numpy import *

# bltin_sum = np.sum

In [2]:
# Functions

# def add(a, b): return a + b

Vector = List[float]

def vector_sum(vectors):
    """Sums all corresponding elements"""
    # Check that vectors is not empty
    assert vectors, "no vectors provided!"

    # Check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes!"

    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
            for i in range(num_elements)]

def scalar_multiply(c , v):
    """Multiplies every element by c"""
    return [c * v_i for v_i in v]

def vector_mean(vectors):
    """Computes the element-wise average"""
    n = len(vectors)
    m = sum(vectors,axis=0)
    vec_mean = np.multiply(1/n,m)
    return vec_mean

def de_mean(xs):
    """Translate xs by subtracting its mean (so the result has mean 0)"""
    x_bar = np.mean(xs)
    d_mean = [x - x_bar for x in xs]
    return d_mean

def dot(v, w):
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), "vectors must be same length"

#     return np.sum(v_i * w_i for v_i, w_i in zip(v, w))
#     gen = 
    return np.sum(np.fromiter((v_i * w_i for v_i, w_i in zip(v, w)),float))

def sum_of_squares(v):
    """Returns v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

def variance(xs):
    """Almost the average squared deviation from the mean"""
    assert len(xs) >= 2, "variance requires at least two elements"

    n = len(xs)
    deviations = de_mean(xs)
    vari = sum_of_squares(deviations)/(n-1)
    return vari

# Standard deviation                        
def standard_deviation(xs):
    """The standard deviation is the square root of the variance"""
    std_dev = np.sqrt(variance(xs)) 
    return std_dev

def scale(data):
    """returns the mean and standard deviation for each position"""
    dim = data.shape[0]
    
    # Vector Mean
#     n = len(data)
#     m = np.sum(data,axis=0)
#     means = np.multiply(1/n,m)
    means = vector_mean(data)
    
    # Standard Deviaiton
    stdevs = [standard_deviation([vector[i] for vector in data])
              for i in range(dim)]
    return means,stdevs

def rescale(data):
    """
    Rescales the input data so that each position has
    mean 0 and standard deviation 1. (Leaves a position
    as is if its standard deviation is 0.)
    """
    dim = data.shape[0]
    means, stdevs = scale(data)
    
    means = list(means)
    stdevs = list(stdevs)

    # Make a copy of each vector
    rescaled = [v[:] for v in data]
    v0 = []
    for v in rescaled:
        v = list(v)
        for i in range(dim):
            if stdevs[i] > 0:
                v[i] = (v[i] - means[i]) / stdevs[i]
        v0.append(v)

    return v0

def gradient_step(v, gradient, step_size):
    """Moves `step_size` in the `gradient` direction from `v`"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    grad_step = np.add(v,step)
    return grad_step

# def predict(alpha, beta, x_i):
#     pred = beta * x_i + alpha
#     return pred

# def error(x, y, beta):
#     """
#     The error from predicting beta * x_i + alpha
#     when the actual value is y_i
#     """
#     err_fin = predict(alpha, beta, x_i) - y_i
#     return err_fin

def predict(x, beta):
    """assumes that the first element of x is 1"""
    return dot(x, beta)

def error(x, y, beta):
    return predict(x, beta) - y 

def sqerror_gradient(x, y, beta):
    err = error(x, y, beta)
    err_fin = [2 * err * x_i for x_i in x]
    return err_fin

def least_squares_fit(xs, ys, learning_rate = 0.001, num_steps = 1000, batch_size = 1):
    """
    Find the beta that minimizes the sum of squared errors
    assuming the model y = dot(x, beta).
    """
    # Start with a random guess
    guess = [np.random.random() for _ in xs[0]]

    for _ in tqdm.trange(num_steps, desc="least squares fit"):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start+batch_size]
            batch_ys = ys[start:start+batch_size]

            gradient = vector_mean([sqerror_gradient(x, y, guess)
                                    for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)

    return guess

def logistic(x):
    return 1.0 / (1 + math.exp(-x))

def logistic_prime(x):
    y = logistic(x)
    return y * (1 - y)

def _negative_log_likelihood(x, y, beta):
    """The negative log likelihood for one data point""" 
    if y == 1:
        return -math.log(logistic(dot(x, beta)))
    else:
        return -math.log(1 - logistic(dot(x, beta)))
    
def negative_log_likelihood(xs, ys, beta):
    return sum(_negative_log_likelihood(x, y, beta)
               for x, y in zip(xs, ys))

def _negative_log_partial_j(x, y, beta, j):
    """
    The jth partial derivative for one data point.
    Here i is the index of the data point.
    """
    return -(y - logistic(dot(x, beta))) * x[j]

def _negative_log_gradient(x, y, beta):
    """
    The gradient for one data point.
    """
    return [_negative_log_partial_j(x, y, beta, j)
            for j in range(len(beta))]

def negative_log_gradient(xs, ys,beta):
    return vector_sum([_negative_log_gradient(x, y, beta)
                       for x, y in zip(xs, ys)])

def split_data(data, prob):
    """Split data into fractions [prob, 1 - prob]"""
    data = data[:]                    # Make a shallow copy
    random.shuffle(data)              # because shuffle modifies the list.
    cut = int(len(data) * prob)       # Use prob to find a cutoff
    return data[:cut], data[cut:]     # and split the shuffled list there.

def train_test_split(xs, ys, test_pct):
     # Generate the indices and split them
    idxs = [i for i in range(len(xs))]
    train_idxs, test_idxs = split_data(idxs, 1 - test_pct)

    return ([xs[i] for i in train_idxs],  # x_train 
            [xs[i] for i in test_idxs],   # x_test
            [ys[i] for i in train_idxs],  # y_train
            [ys[i] for i in test_idxs])   # y_test
                                                                
def step_function(x: float) -> float:
    return 1.0 if x >= 0 else 0.0

def sigmoid(t: float) -> float: 
    return 1 / (1 + math.exp(-t))

# Gradient Descent - step
def gradient_step(v: Vector, gradient: Vector, step_size: float):
    """Moves `step_size` in the `gradient` direction from `v`"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

def sum_of_squares_gradient(v: Vector) -> Vector:
    return [2 * v_i for v_i in v]

def squared_distance(v: Vector, w: Vector) -> float:
    """Computes (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """Computes the distance between v and w"""
    return math.sqrt(squared_distance(v, w))

In [3]:
# Neural Netwroks Functions

def perceptron_output(weights, bias, x):
    """Returns 1 if the perceptron 'fires', 0 if not"""
    calculation = dot(weights, x) + bias
    return step_function(calculation)

Vector = List[float]


In [4]:
# Feed-Forward Neural Networks
def neuron_output(weights, inputs):
    # weights includes the bias term, inputs includes a 1
    return sigmoid(dot(weights, inputs))

def feed_forward(neural_network, input_vector):
    """
    Feeds the input vector through the neural network.
    Returns the outputs of all layers (not just the last one).
    """
    outputs: List[Vector] = []

    for layer in neural_network:
        input_with_bias = input_vector + [1]              # Add a constant.
        output = [neuron_output(neuron, input_with_bias)  # Compute the output
                  for neuron in layer]                    # for each neuron.
        outputs.append(output)                            # Add to results.

        # Then the input to the next layer is the output of this one
        input_vector = output

    return outputs

xor_network = [# hidden layer
               [[20., 20, -30],      # 'and' neuron
                [20., 20, -10]],     # 'or'  neuron
               # output layer
               [[-60., 60, -30]]]    # '2nd input but not 1st input' neuron

abcd = feed_forward(xor_network, [1,0])[-1][0]
abcde = feed_forward(xor_network, [1,0])
abcde


[[4.5397868702434395e-05, 0.9999546021312976], [0.9999999999999059]]

In [5]:
xor_network

[[[20.0, 20, -30], [20.0, 20, -10]], [[-60.0, 60, -30]]]

In [6]:
# def sqerror_gradients(network, input_vector, target_vector):
    
# #   """
# #   Given a neural network, an input vector, and a target vector,
# #   make a prediction and compute the gradient of the squared error
# #   loss with respect to the neuron weights.
# #   """
#      # forward pass
#      hidden_outputs, outputs = feed_forward(network, input_vector)
#      # gradients with respect to output neuron pre-activation outputs
#      output_deltas = [output * (1 - output) * (output - target)
#                       for output, target in zip(outputs, target_vector)]
#      # gradients with respect to output neuron weights
#      output_grads = [[output_deltas[i] * hidden_output
#                       for hidden_output in hidden_outputs + [1]]
#                      for i, output_neuron in enumerate(network[-1])]
#      # gradients with respect to hidden neuron pre-activation outputs
#      hidden_deltas = [hidden_output * (1 - hidden_output) *
#                           dot(output_deltas, [n[i] for n in network[-1]])
#                       for i, hidden_output in enumerate(hidden_outputs)]
#      # gradients with respect to hidden neuron weights
#      hidden_grads = [[hidden_deltas[i] * input for input in input_vector + [1]]
#                      for i, hidden_neuron in enumerate(network[0])]
        
# return [output_grads]

In [7]:
def sqerror_gradients(network, input_vector, target_vector):
    
# Given a neural network, an input vector, and a target vector,
# make a prediction and compute the gradient of the squared error
# loss with respect to the neuron weights.

     # forward pass
     hidden_outputs, outputs = feed_forward(network, input_vector)
     # gradients with respect to output neuron pre-activation outputs
     output_deltas = [output * (1 - output) * (output - target)
                      for output, target in zip(outputs, target_vector)]
     # gradients with respect to output neuron weights
     output_grads = [[output_deltas[i] * hidden_output
                      for hidden_output in hidden_outputs + [1]]
                     for i, output_neuron in enumerate(network[-1])]
     # gradients with respect to hidden neuron pre-activation outputs
     hidden_deltas = [hidden_output * (1 - hidden_output) *
                          dot(output_deltas, [n[i] for n in network[-1]])
                      for i, hidden_output in enumerate(hidden_outputs)]
    # gradients with respect to hidden neuron weights
     hidden_grads = [[hidden_deltas[i] * input for input in input_vector + [1]]
                     for i, hidden_neuron in enumerate(network[0])]
     return [hidden_grads,output_grads]


In [8]:
# hidden_grads

In [9]:
random.seed(0) 
 
# training data
xs = [[0., 0], [0., 1], [1., 0], [1., 1]]
ys = [[0.], [1.], [1.], [0.]] 

In [10]:
# start with random weights
network = [ # hidden layer: 2 inputs -> 2 outputs
            [[random.random() for _ in range(2 + 1)],   # 1st hidden neuron
             [random.random() for _ in range(2 + 1)]],  # 2nd hidden neuron
            # output layer: 2 inputs -> 1 output
            [[random.random() for _ in range(2 + 1)]]   # 1st output neuron
          ]

In [11]:
learning_rate = 1.0

for epoch in tqdm.trange(20000, desc="neural net for xor"):
    for x, y in zip(xs, ys):
        gradients = sqerror_gradients(network, x, y)

        # Take a gradient step for each neuron in each layer
        network = [[gradient_step(neuron, grad, -learning_rate)
                    for neuron, grad in zip(layer, layer_grad)]
                   for layer, layer_grad in zip(network, gradients)]
 

neural net for xor: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:06<00:00, 2977.04it/s]


In [12]:
network

[[array([-8.21157876, -8.27217603,  3.32324842]),
  array([-5.09668499, -5.10005374,  7.57504465])],
 [array([-10.91345369,  10.98905957,  -5.28689396])]]

In [13]:
test_ff = feed_forward(network, [0, 1])[-1][0] 
test_ff

0.9916095989155516

# Example Fizz Buzz

In [14]:
def fizz_buzz_encode(x):
    if x % 15 == 0:
        return [0, 0, 0, 1]
    elif x % 5 == 0:
        return [0, 0, 1, 0]
    elif x % 3 == 0:
        return [0, 1, 0, 0]
    else:
        return [1, 0, 0, 0]

f_buzz_0 = fizz_buzz_encode(2)
f_buzz_0


[1, 0, 0, 0]

In [15]:
def binary_encode(x):
    binary: List[float] = []

    for i in range(10):
        binary.append(x % 2)
        x = x // 2

    return binary

binary_encode(101)

[1, 0, 1, 0, 0, 1, 1, 0, 0, 0]

In [16]:
xs = [binary_encode(n) for n in range(101, 1024)]
ys = [fizz_buzz_encode(n) for n in range(101, 1024)]
# xs = np.array([[1,0,1,0],[1,0,1,1],[0,1,0,1]])
# ys = np.array([[1],[1],[0]])

In [17]:
NUM_HIDDEN = 25

network = [ 
    # hidden layer: 10 inputs -> NUM_HIDDEN outputs
    [[random.random() for _ in range(10 + 1)] for _ in range(NUM_HIDDEN)],

    # output_layer: NUM_HIDDEN inputs -> 4 outputs
    [[random.random() for _ in range(NUM_HIDDEN + 1)] for _ in range(4)]
]

In [18]:

learning_rate = 1.0

with tqdm.trange(500) as t:
    for epoch in t:
        epoch_loss = 0.0

        for x, y in zip(xs, ys):
            predicted = feed_forward(network, x)[-1]
            epoch_loss += squared_distance(predicted, y)
            gradients = sqerror_gradients(network, x, y)

            # Take a gradient step for each neuron in each layer
            network = [[gradient_step(neuron, grad, -learning_rate)
                        for neuron, grad in zip(layer, layer_grad)]
                    for layer, layer_grad in zip(network, gradients)]

        t.set_description(f"fizz buzz (loss: {epoch_loss:.2f})")

fizz buzz (loss: 155.77): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [09:42<00:00,  1.16s/it]


In [19]:
network
epoch_loss
gradients

for x, y in zip(xs, ys):
    print(x)
    print(y)

[1, 0, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0, 0, 1, 1, 0, 0, 0]
[0, 1, 0, 0]
[1, 1, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 1, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 1]
[0, 1, 0, 1, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[1, 1, 0, 1, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 1, 1, 0, 1, 1, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 1, 1, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 1, 0, 1, 1, 0, 0, 0]
[0, 0, 1, 0]
[1, 1, 1, 1, 0, 1, 1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
[0, 1, 0, 0]
[1, 1, 0, 0, 1, 1, 1, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 1, 0, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 1, 0, 0, 0]
[0, 1, 0, 0]
[0, 1, 1, 0, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[1, 1, 1, 0, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 1]
[1, 0, 0, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0]
[1, 1, 0, 1, 1, 1, 1, 0, 0, 0]
[

In [20]:
def argmax(xs: list) -> int:
    """Returns the index of the largest value"""
    return max(range(len(xs)), key=lambda i: xs[i])

xs = [-1, 10, 5, 20, -3]
argmax_tst = argmax(xs)


In [21]:
num_correct = 0

for n in range(1, 101):
    x = binary_encode(n)
    predicted = argmax(feed_forward(network, x)[-1])
    actual = argmax(fizz_buzz_encode(n))
    labels = [str(n), "fizz", "buzz", "fizzbuzz"]
    print(n, labels[predicted], labels[actual])

    if predicted == actual:
        num_correct += 1

print(num_correct, "/", 100)

1 1 1
2 2 2
3 fizz fizz
4 fizz 4
5 buzz buzz
6 fizz fizz
7 7 7
8 fizz 8
9 fizz fizz
10 buzz buzz
11 11 11
12 fizz fizz
13 13 13
14 14 14
15 15 fizzbuzz
16 16 16
17 17 17
18 fizz fizz
19 19 19
20 buzz buzz
21 fizz fizz
22 22 22
23 23 23
24 fizz fizz
25 buzz buzz
26 26 26
27 fizz fizz
28 fizz 28
29 29 29
30 30 fizzbuzz
31 31 31
32 32 32
33 fizz fizz
34 34 34
35 buzz buzz
36 fizz fizz
37 37 37
38 38 38
39 fizz fizz
40 buzz buzz
41 41 41
42 buzz fizz
43 43 43
44 44 44
45 45 fizzbuzz
46 46 46
47 47 47
48 fizz fizz
49 49 49
50 buzz buzz
51 fizz fizz
52 52 52
53 53 53
54 fizz fizz
55 fizz buzz
56 56 56
57 fizz fizz
58 58 58
59 59 59
60 60 fizzbuzz
61 61 61
62 62 62
63 fizz fizz
64 fizz 64
65 fizz buzz
66 fizz fizz
67 67 67
68 fizz 68
69 fizz fizz
70 buzz buzz
71 71 71
72 fizz fizz
73 73 73
74 74 74
75 75 fizzbuzz
76 76 76
77 77 77
78 fizz fizz
79 79 79
80 fizz buzz
81 fizz fizz
82 82 82
83 83 83
84 fizz fizz
85 buzz buzz
86 86 86
87 fizz fizz
88 88 88
89 89 89
90 90 fizzbuzz
91 91 91
92 92 92

In [22]:
n = 99
x = binary_encode(n)
predicted = argmax(feed_forward(network, x)[-1])
actual = argmax(fizz_buzz_encode(n))
labels = [str(n), "fizz", "buzz", "fizzbuzz"]
print(predicted)
print(actual)
print(n, labels[predicted], labels[actual])

1
1
99 fizz fizz
